# Import Required Libraries

In [53]:
import os
import warnings
warnings.filterwarnings("ignore")

from getpass import getpass
os.environ["HF_TOKEN"] = getpass("Enter the Huggingface Token: ")

In [54]:
from datasets import load_dataset
from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM)
from transformers import (Trainer, 
                         TrainingArguments)

from IPython.display import Markdown, display

In [55]:
import torch
if torch.cuda.is_available():
    print(f"GPU Name : {torch.cuda.get_device_name(0)}")
    print(f"GPU Version : {torch.version.cuda}")


device = "cuda" if torch.cuda.is_available() else "cpu"

GPU Name : Tesla T4
GPU Version : 12.6


# Load the huggingface dataset

In [56]:
dataset = load_dataset(
    path = "lamini/taylor_swift", 
)

# We are dropping these columns and we recreate them as per our use case
dataset = dataset.remove_columns(["input_ids", "attention_mask", "labels"])
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 783
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 87
    })
})

In [57]:
def questionAnswerPairs(example):
    """
        We will be using prompt template for our modelling

    """

    prompt_template = f"""
        Question : 
                {example["question"]}

        Answer : 
                {example["answer"]}
    
    """

    example["prompt"] = prompt_template
 
    return example

# Generate the prompt 
dataset= dataset.map(questionAnswerPairs)

In [58]:
display(Markdown(dataset["train"][0]["prompt"]))


        Question : 
                What is the controversy surrounding Taylor Swift's music and how has it impacted her career?

        Answer : 
                Taylor Swift has been involved in several controversies throughout her career, including her feud with Kanye West and Kim Kardashian, her lawsuit against a radio DJ who allegedly groped her, and her recent feud with Scooter Braun. These controversies have impacted her career in several ways. First, they have made her a more polarizing figure in the music industry, with some fans supporting her and others criticizing her. Second, they have led to a decrease in her popularity among some listeners, particularly those who do not agree with her political views or her actions in the feuds. Finally, they have led to a decrease of her music being played on some radio stations, which has impacted her ability to reach new audiences
    
    

# 2. Tokenization - Instruction Style Question Answer Pairs

In [59]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

In [60]:
tokenizer.eos_token_id

0

In [61]:
def tokenize_instruction_qa(example, tokenizer, max_len = 2048):
    # Build prompt
    prompt = (
        f"Question: {example['question']} \n"
        f"Answer:"
    )

    answer = example["answer"]

    # Tokenize prompt and answer
    tok_prompt = tokenizer(prompt, add_special_tokens = True)
    tok_answer = tokenizer(answer, add_special_tokens = True)

    input_ids = tok_prompt["input_ids"] + tok_answer["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = [1]*(len(input_ids))

    # Prepare labels (mask prompt tokens)
    #labels = [-100] * len(tok_prompt["input_ids"]) + tok_answer["input_ids"] + [tokenizer.eos_token_id]
    labels = input_ids


    # Truncate if needed
    if len(input_ids) > max_len:
        input_ids = input_ids[-max_len:]
        attention_mask = attention_mask[-max_len:]
        labels = labels[-max_len:]

    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels
    }


In [62]:
dataset = dataset.map(lambda x : tokenize_instruction_qa(example = x, tokenizer = tokenizer))

Map:   0%|          | 0/783 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

# 3. Load the base Model 

In [63]:
base_model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-70m",
    device_map = "auto"
)

base_model.to(device) # load the base model to the gpu

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [64]:
def inference(test, model, tokenizer, max_input_tokens = 1000, max_output_tokens= 100):
    # Tokenize the inference text
    input_ids = tokenizer.encode(
        text, 
        return_tensors = "pt",
        truncation = True,
        max_length = max_input_tokens
    ).to(device)
    
    # Generate the Predictions / Next Token
    predicted_tokens =  model.generate(
        input_ids = input_ids, 
        max_length = max_output_tokens
    )

    # Decode the predicted tokens
    predicted_text =tokenizer.batch_decode(predicted_tokens, skip_special_tokens=True)

    return predicted_text


In [65]:
text = "What is the controversy surrounding Taylor Swift's music and how has it impacted her career?"
display(Markdown(inference(text, base_model, tokenizer)[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is the controversy surrounding Taylor Swift's music and how has it impacted her career?

The controversy surrounding the song "The New York Times" is a great example of how the song is used to describe the song. The song is a song that is used to describe the song's lyrics. The song is a song that is used to describe the song's lyrics. The song is a song that is used to describe the song's lyrics. The song is a song that is used to describe

In [66]:
# We can clearly observe the response is getting repeated and not up to the mark, so lets how it's gonna answer post fine tuning the model ?

In [67]:
train_dataset, test_dataset = dataset["train"], dataset["test"]


# Just for the similicity we have divided the dataset into train, test dataset 
train_dataset = train_dataset.select_columns(["input_ids", "labels"])
test_dataset = test_dataset.select_columns(["input_ids", "labels"])

# 5. Define the Trainer Arguments

In [77]:
training_args = TrainingArguments(
    # Core training configuration
    seed = 40, # Random seed for initializing, ensuring reproduction
    optim = "adamw_torch", # Optimizer, here it's a AdamW implemented by pytorch
    max_steps = 5000, # Number of maximum training steps 
    per_device_train_batch_size = 1, # Batch size per device during training 

    # Other training configuration
    learning_rate = 5e-05, # Initial learning rate for the optimizer
    weight_decay = 0, # Weight decay
    warmup_steps = 10, # Number of steps for the learning rate warmup phase 
    gradient_accumulation_steps = 2,  # Number of steps to accumulate gradients before updating
    bf16 = False, # Use brain float16 for training on supported hardware
    output_dir = "TalorSwiftFineTunedModel", # output dir location

    # logging configuration
    logging_steps = 3, # Frequency of logging training information
    report_to= None # Destination for logging (e.g.., WandB, TensorBoard)
)

In [78]:
# Initialize HuggingFace Trainer 

trainer = Trainer(
    base_model, 
    training_args, 
    train_dataset = train_dataset, 
    eval_dataset = test_dataset, 
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
3,1.922500
6,1.249200
9,1.061200
12,1.273000
15,2.009500
18,2.186800
21,1.679200
24,2.364900
27,1.803100
30,1.656500


TrainOutput(global_step=5000, training_loss=0.7791307537019253, metrics={'train_runtime': 317.5453, 'train_samples_per_second': 31.492, 'train_steps_per_second': 15.746, 'total_flos': 177885008265216.0, 'train_loss': 0.7791307537019253, 'epoch': 12.756066411238825})

# Save Model Locally

In [79]:
save_dir = 'TaylorShiftFineTunedModel/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: TaylorShiftFineTunedModel/final


In [80]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, 
                                                            local_files_only=True)

finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [81]:
def memory_info():
    " Function to check the total memory available"
    total_memory = torch.cuda.get_device_properties(device).total_memory # Total memory of the GPUs
    allocated_memory = torch.cuda.memory_allocated(device) # currently allocated memory by tensors
    reserved_memory = torch.cuda.memory_reserved(device) # memory reserved by the caching allocator 

    # Free memory (within reserved)
    free_mem = reserved_memory - allocated_memory

    print(f"Total GPU Memory: {total_memory/ 1024**3:.2f} GB")
    print(f"Allocated GPU Memory: {allocated_memory / 1024**3:.2f} GB")
    print(f"Reserved Memory: {reserved_memory / 1024**3:.2f} GB")
    print(f"Free (within reserved): {free_mem / 1024**3:.2f} GB")

memory_info()

Total GPU Memory: 14.74 GB
Allocated GPU Memory: 2.53 GB
Reserved Memory: 2.87 GB
Free (within reserved): 0.34 GB


# Evaluation at Max Steps = 300

In [74]:
text = "What is the controversy surrounding Taylor Swift's music and how has it impacted her career?"
display(Markdown(inference(text, finetuned_slightly_model, tokenizer)[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is the controversy surrounding Taylor Swift's music and how has it impacted her career? 
Answer:Taylor Swift's music and how has it impacted her career. The controversy surrounding Taylor Swift's music and how has it impacted her career? 
As a Swiftie Bot, Swift has been a part of Swiftie's life and her career. The controversy surrounding her music and how has it impacted her career. The controversy surrounding her music and her career has affected her career and her career. The

# Evaluation at Max Steps = 5000

In [83]:
text = "What is the controversy surrounding Taylor Swift's music and how has it impacted her career?"
display(Markdown(inference(text, finetuned_slightly_model, tokenizer)[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[ 1276,   310,   253, 16305,  8704, 11276, 24619,   434,  3440,   285,
           849,   556,   352, 27857,   617,  5249,    32]], device='cuda:0')
tensor([[ 1276,   310,   253, 16305,  8704, 11276, 24619,   434,  3440,   285,
           849,   556,   352, 27857,   617,  5249,    32,   209,   187, 32869,
            27, 37979, 24619,   556,   644,  3206,   275,  2067,  9474,   447,
          4768,   617,  5249,    13,  1690,   617, 40157,   342,   611,  1279,
            70,  4255,   285, 10766,   611,   472,  1225,   757,    13,   617,
         15091,  1411,   247,  5553, 22533,   665, 14163,   305,  1658,   264,
           617,    13,   285,   617,  3332, 40157,   342,  1810, 37449, 47907,
            15,  2053,  9474,   447,   452, 27857,   617,  5249,   275,  2067,
          4088,    15,  3973,    13,   597,   452,  1160,   617,   247,   625,
          6994,  3006,  4677,   275,   253,  3440,  4491,    13,   342,   690]],
       device='cuda:0')


What is the controversy surrounding Taylor Swift's music and how has it impacted her career? 
Answer:Taylor Swift has been involved in several controversies throughout her career, including her feud with Kanye West and Kim Kardashian, her lawsuit against a radio DJ who allegedly groped her, and her recent feud with Scooter Braun. These controversies have impacted her career in several ways. First, they have made her a more polarizing figure in the music industry, with some

In [ ]:
# This is a great observation as we increase the number of steps the model results are getting better and cohorted
# See the difference between the models at steps 300 and steps 5000 the model has learnt information more to provide more clearly and accurate 